<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>


## Introduction


We will build an image classifier using the VGG16 pre-trained model, and will evaluate it and compare its performance to the ResNet50 pre-trained model.

<a id="item41"></a>

## Data

 https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [1]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-07-18 19:49:47--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

100%[======================================>] 261,483,817 41.9MB/s   in 6.6s   

2020-07-18 19:49:54 (38.1 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

Data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

Design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, we import the model <code>VGG16</code> from <code>keras.applications</code>.

Build your classifier as follows:
1. Import libraries, modules.Import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

In [43]:
# import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [4]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [7]:

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [8]:
model = Sequential()

In [9]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [10]:
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.layers

In [12]:
model.layers[0].layers

In [13]:
model.layers[0].trainable = False

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [17]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
301/301 [==============================] - 10864s 36s/step - loss: 0.1127 - acc: 0.9619 - val_loss: 0.0287 - val_acc: 0.9941
Epoch 2/2
301/301 [==============================] - 10849s 36s/step - loss: 0.0227 - acc: 0.9953 - val_loss: 0.0167 - val_acc: 0.9963


In [44]:
model.save('classifier_vgg_model.h5')

<a id="item43"></a>

## Part 2

Evaluate deep learning models on a test data.

1. Load saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. Pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate models on the test data, by passing the above ImageDataGenerator as an argument.
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


In [ ]:
from tensorflow.keras.models import load_model

import os
import numpy as np
import matplotlib.pyplot as plt

model_ResNet = load_model('classifier_resnet_model.h5')

In [ ]:
model_ResNet.save('classifier_resnet1_model.h5')

In [62]:
test_data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle= False)

Found 500 images belonging to 2 classes.


In [70]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [74]:
# VGG16 model
VGG16_model_eval = model.evaluate_generator(test_generator, len(test_generator))
print("VGG16 Model performance : ", VGG16_model_eval[1])

VGG16 Model performance :  0.996


In [ ]:
# ResNet50 model
res_model_eval = model_ResNet.evaluate_generator(test_generator, len(test_generator))
print("ResNet50 Model performance : ", res_model_eval[1])

<a id="item44"></a>

## Part 3

Predict whether the images in the test data are images of cracked concrete or not. 

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. 
2. Report the class predictions of the first five images in the test set. 

In [ ]:
# VGG16 model
vgg_predict = model.predict_generator(test_generator, len(test_generator))
vgg_predict

In [83]:
def model_labels(arr):
  if(arr[0]>arr[1]):
    return "Negative"
  else:
    return "Positive"

In [84]:
for i in range(5):
  print(model_labels(vgg_predict[i]))

Negative
Negative
Negative
Negative
Negative


In [ ]:
# Resnet50 model prediction in test set
for i in range(5):
res_predict = model_resnet.predict_generator(test_generator, len(test_generator))
print("ResNet50 first five images:",(model_labels(res_pred[i])